# Лабораторная работа №1 

### Задание 1

Найти набор данных (датасет) для классификации удовлетворяющий следующим условиям: более 10 000 строк, более 20 столбцов, разные типы в столбцах, обязательно наличие целевого признака (таргета).(Документация датасета находится в корне, в файле datasheet.pdf)

### Задание 2

Провести классификацию найденного датасета, методом к- ближайших соседей. В формате Markdown писать пояснения. Объяснить почему были выбраны именно такие гиперпараметры, была ли перекрестная проверка, и т.д.

### Шаг 1. Подготовка данных

In [28]:
import pandas as pd
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.preprocessing import StandardScaler
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score, classification_report

# Загрузка данных
df = pd.read_csv("./Base_min_shuffled.csv")

print(df)

       fraud_bool  income  name_email_similarity  prev_address_months_count  \
0               0     0.5               0.041967                         11   
1               0     0.8               0.865305                         -1   
2               0     0.9               0.788227                         -1   
3               0     0.7               0.182315                         -1   
4               0     0.9               0.718332                         -1   
...           ...     ...                    ...                        ...   
11995           1     0.4               0.369884                         -1   
11996           0     0.3               0.679246                         -1   
11997           0     0.8               0.784484                         -1   
11998           0     0.6               0.509644                         -1   
11999           0     0.6               0.302013                         -1   

       current_address_months_count  customer_age  

### Шаг 2. Разделение данных и нормализация

In [29]:
# Разделение данных на признаки (X) и целевую переменную (y)
y = df["fraud_bool"]
X = df.drop("fraud_bool", axis=1)


columns_to_drop = ['payment_type', 'employment_status', 'source', 'device_os','housing_status']

# Удаление указанных столбцов
df = df.drop(columns=columns_to_drop)

y = df["fraud_bool"]
X = df.drop("fraud_bool", axis=1)

# Разделение на тренировочные и тестовые данные
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Стандартизация данных
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

print(f"Размер обучающего набора: {X_train.shape}")
print(f"Размер тестового набора: {X_test.shape}")


Размер обучающего набора: (9600, 26)
Размер тестового набора: (2400, 26)


###  Подбор гиперпараметров и обучение модели

In [30]:
knn = KNeighborsClassifier()

param_grid = {'n_neighbors': list(range(2, 11)), 'metric': ['euclidean', 'manhattan', 'chebyshev', 'minkowski']}

grid = GridSearchCV(knn, param_grid, refit=True, verbose=3, cv=5)
# Обучение модели
grid.fit(X_train, y_train)

best_params = grid.best_params_
best_knn = grid.best_estimator_

Fitting 5 folds for each of 36 candidates, totalling 180 fits
[CV 1/5] END ...metric=euclidean, n_neighbors=2;, score=0.829 total time=   0.0s
[CV 2/5] END ...metric=euclidean, n_neighbors=2;, score=0.840 total time=   0.1s
[CV 3/5] END ...metric=euclidean, n_neighbors=2;, score=0.829 total time=   0.0s
[CV 4/5] END ...metric=euclidean, n_neighbors=2;, score=0.834 total time=   0.0s
[CV 5/5] END ...metric=euclidean, n_neighbors=2;, score=0.836 total time=   0.0s
[CV 1/5] END ...metric=euclidean, n_neighbors=3;, score=0.827 total time=   0.0s
[CV 2/5] END ...metric=euclidean, n_neighbors=3;, score=0.833 total time=   0.0s
[CV 3/5] END ...metric=euclidean, n_neighbors=3;, score=0.824 total time=   0.0s
[CV 4/5] END ...metric=euclidean, n_neighbors=3;, score=0.826 total time=   0.0s
[CV 5/5] END ...metric=euclidean, n_neighbors=3;, score=0.817 total time=   0.0s
[CV 1/5] END ...metric=euclidean, n_neighbors=4;, score=0.838 total time=   0.0s
[CV 2/5] END ...metric=euclidean, n_neighbors=4

### Обоснование выбора гиперпараметров

In [31]:
# Инициализация и обучение модели методом ближайших соседей
predictions = best_knn.predict(X_test)

# Предсказание на тестовых данных

# Оценка качества модели
accuracy = accuracy_score(y_test, predictions)
report = classification_report(y_test, predictions)

print(f"Accuracy: {accuracy}")
print(f"\nЛучшие параметры: {best_params}")
print("Classification Report:\n", report)

Accuracy: 0.86125

Лучшие параметры: {'metric': 'euclidean', 'n_neighbors': 8}
Classification Report:
               precision    recall  f1-score   support

           0       0.88      0.97      0.92      2038
           1       0.60      0.24      0.34       362

    accuracy                           0.86      2400
   macro avg       0.74      0.60      0.63      2400
weighted avg       0.84      0.86      0.83      2400

